In [1]:
# !pip install -r requirements.txt

import warnings
import pandas as pd
import logging
import os
import sys
from utils.LabelsDict import tickers
#from utils.load_data import *

warnings.filterwarnings("ignore", category=UserWarning, message="Unable to import Axes3D")

from utils.portfolio import Portfolio

# Создаем директорию для логов если её нет
os.makedirs('logs', exist_ok=True)
from utils.logger import Logger
log = Logger(__name__)

from pycbrf.toolbox import ExchangeRates, Banks

In [3]:
# No Logger class to reset; handlers are cleared above if needed

# Define tickers list
tickers_list = [
    'GAZP', 'LKOH', 'ROSN',
    'SBER', 'VTBR', 'MOEX',
    'GMKN', 'NLMK', 'RUAL',
    'MTSS', 'RTKM', 'TTLK',
    'MGNT', 'LNTA', 'FESH',
]

# Recreate the Portfolio instance with the updated code
calc = Portfolio(
    dt_calc='2025-11-30',
    dt_start='2019-11-03',
    stocks_step=10,
    tickers_list=tickers_list
)

In [4]:
# 1. Логирование системной информации
calc.log_system_info()

2025-12-06 17:50:31,940:utils.portfolio:INFO: Configuration Parameters
Parameter                                                                                            Value
Calculation Date                                                                                2025-11-30
Start Date                                                                                      2019-11-03
Stocks Step                                                                                             10
Tickers Count                                                                                           15
Tickers           GAZP, LKOH, ROSN, SBER, VTBR, MOEX, GMKN, NLMK, RUAL, MTSS, RTKM, TTLK, MGNT, LNTA, FESH


In [11]:
# 2. Загрузка рыночных данных
calc.load_stock_data(
    use_backup_data=True,
    create_backup=False
)

2025-12-06 17:51:40,909:utils.portfolio:INFO: Backup loaded. Last date: 2025-12-01
2025-12-06 17:51:40,914:utils.portfolio:INFO: Backup is up to date.


In [13]:
# 3. Загрузка мультипликаторов
calc = calc.load_multipliers()

2025-12-06 17:51:51,128:utils.load_data:INFO: Updated multipliers backup for GAZP
2025-12-06 17:51:51,250:utils.load_data:INFO: Updated multipliers backup for LKOH
2025-12-06 17:51:51,398:utils.load_data:INFO: Updated multipliers backup for ROSN
2025-12-06 17:51:51,532:utils.load_data:INFO: Updated multipliers backup for SBER
2025-12-06 17:51:51,658:utils.load_data:INFO: Updated multipliers backup for VTBR
2025-12-06 17:51:51,789:utils.load_data:INFO: Updated multipliers backup for MOEX
2025-12-06 17:51:52,018:utils.load_data:INFO: Updated multipliers backup for GMKN
2025-12-06 17:51:52,217:utils.load_data:INFO: Updated multipliers backup for NLMK
2025-12-06 17:51:52,401:utils.load_data:INFO: Updated multipliers backup for RUAL
2025-12-06 17:51:52,613:utils.load_data:INFO: Updated multipliers backup for MTSS
2025-12-06 17:51:52,808:utils.load_data:INFO: Updated multipliers backup for RTKM
2025-12-06 17:51:53,015:utils.load_data:INFO: Updated multipliers backup for TTLK
2025-12-06 17:51

In [6]:
# 4. Создание портфеля (объединение данных)
calc = calc.create_portfolio()

2025-11-30 18:32:02,477:utils.portfolio:INFO: Portfolio created | Companies: 15


In [7]:
# 5. Настройка форматов данных
calc = calc.adjust_portfolio_data_types()

2025-11-30 18:32:02,774:utils.portfolio:INFO: Column types adjusted: ['Долг, млрд руб', 'Капитализация, млрд руб', 'Чистый долг, млрд руб', 'high', 'low', 'close', 'EV/EBITDA', 'P/BV', 'P/E', 'P/S', 'open', 'Долг/EBITDA']


In [8]:
# 6. Загрузка макропараметров
calc = calc.add_macro_data()

2025-11-30 18:32:03,382:utils.load_data:INFO: Backup file for usd/rub exchange rates was updated.New dates range: 2019-11-03 : 2025-05-31
2025-11-30 18:32:03,394:utils.portfolio:INFO: Macro indicators added: Interest rate, Unemployment, Inflation, USD/RUB


In [9]:
# 7. Обработка пропусков в данных
calc = calc.fill_missing_values()

2025-11-30 18:32:05,863:utils.portfolio:INFO: ============================================================
2025-11-30 18:32:05,865:utils.portfolio:INFO: Missing Values Summary (Before Filling)
2025-11-30 18:32:05,868:utils.portfolio:INFO: ------------------------------------------------------------
2025-11-30 18:32:05,869:utils.portfolio:INFO: debt                           | 33.57%
2025-11-30 18:32:05,870:utils.portfolio:INFO: capitalization                 | 12.98%
2025-11-30 18:32:05,872:utils.portfolio:INFO: rubusd_exchange_rate           | 0.00%
2025-11-30 18:32:05,873:utils.portfolio:INFO: ============================================================
2025-11-30 18:32:05,874:utils.portfolio:INFO: Missing values filled in: ['debt', 'capitalization', 'rubusd_exchange_rate']


In [10]:
# 8. (Опционально) Дополнительные расчеты и визуализация
# calc.add_dynamic_features()                                   # проставить динамические признаки по дате, например, квартал
# calc.add_merton_pd()                                          # выполнить расчет вероятности дефолта по формуле Мертона
# calc.plot_pd_by_tickers(tickers=tickers_list, verbose=False)  # нарисовать графики для вероятностей дефолта
# calc.plot_stocks(tickers=tickers_list, verbose=False)         # нарисовать динамику котировок акций
# calc.plot_debt_capitalization(verbose=True)                  # нарисовать динамику долга и стоимости активов
# calc.calc_irf(impulses_responses = {                          # расчитать функции импульсного отклика
#     'inflation': 'PD', 'interest_rate': 'PD',
#     'rubusd_exchange_rate': 'PD', 'unemployment_rate': 'PD'
# }, verbose=True)
# calc.plot_correlation_matrix(custom_order = [                 # нарисовать матрицу корреляций
#     'GAZP', 'LKOH', 'ROSN',                               # Нефтегазовая отрасль
#     'SBER', 'VTBR', 'MOEX',                               # Финансовый сектор
#     'GMKN', 'NLMK', 'RUAL',                               # Металлургия
#     'MTSS', 'RTKM', 'TTLK',                               # Телекоммуникации
#     'MGNT', 'LNTA', 'FESH'                                # Розничная торговля
# ], verbose=False)
# calc.calc_macro_connections()                                 # расчитать регрессию для макропараметров
# calc.log_completion()                                         # вывести в лог сообщение об окончании расчета

In [12]:
# Разбили цепочку вызовов на последовательные команды для избежания дублирования выполнения
calc = calc.log_system_info()




In [13]:
calc = calc.load_stock_data(
    use_backup_data=True,                                 # загружать данные из backup копии
    create_backup=False                                   # обновить backup копию
)

2025-11-30 18:32:11,879:utils.portfolio:INFO: Stocks data loaded from backup | Records: 855


In [14]:
calc = calc.load_multipliers()                                   # выделить мультипликаторы из МСФО

2025-11-30 18:32:12,514:utils.portfolio:INFO: Multipliers data loaded | Features: ['ticker', 'year', 'quarter', 'EV/EBITDA', 'P/BV', 'P/E', 'P/FCF', 'P/S', 'Долг, млрд руб', 'Долг/EBITDA', 'Капитализация, млрд руб', 'Чистый долг, млрд руб']


In [ ]:
# 🧪 ТЕСТ: Сравнение логирования из модуля vs напрямую из ноутбука
import logging
from utils.logger import get_logger

print("=" * 80)
print("ТЕСТ 1: Создание логгера НАПРЯМУЮ в ноутбуке (без модуля Portfolio)")
print("=" * 80)

# Создаем логгер напрямую
notebook_logger = get_logger('test.notebook')

print(f"Количество handlers у notebook logger: {len(notebook_logger.handlers)}")
for i, h in enumerate(notebook_logger.handlers):
    print(f"  Handler {i+1}: {type(h).__name__}")

# Логируем 3 сообщения
for i in range(1, 4):
    notebook_logger.info(f"[NOTEBOOK] Test message {i}")

print("\n" + "=" * 80)
print("ТЕСТ 2: Вызов логгера из МОДУЛЯ Portfolio (через calc.logger)")
print("=" * 80)

# Проверяем логгер Portfolio
portfolio_logger = logging.getLogger('utils.portfolio')
print(f"Количество handlers у portfolio logger: {len(portfolio_logger.handlers)}")
for i, h in enumerate(portfolio_logger.handlers):
    print(f"  Handler {i+1}: {type(h).__name__}")

# Логируем через модуль Portfolio
calc.logger.info("[MODULE] Test message from Portfolio module")
calc.logger.info("[MODULE] Second message from Portfolio")
calc.logger.info("[MODULE] Third message from Portfolio")

print("\n" + "=" * 80)
print("ТЕСТ 3: Проверка - это ОДИН И ТОТ ЖЕ объект логгера?")
print("=" * 80)

# Получим логгер с тем же именем
another_logger = get_logger('utils.portfolio')

print(f"calc.logger is another_logger: {calc.logger is another_logger}")
print(f"ID calc.logger: {id(calc.logger)}")
print(f"ID another_logger: {id(another_logger)}")
print(f"Handlers у another_logger: {len(another_logger.handlers)}")

print("\n" + "=" * 80)
print("⚠️ ПРОВЕРЬТЕ ВЫВОД ВЫШЕ:")
print("  - Сколько раз появилось каждое сообщение?")
print("  - Notebook messages должны появиться 1 раз")
print("  - Module messages могут появиться несколько раз если есть дубликаты handlers")
print("=" * 80)


2025-11-11 00:44:22,885:test.notebook:INFO: [NOTEBOOK] Test message 1
2025-11-11 00:44:22,886:test.notebook:INFO: [NOTEBOOK] Test message 2
2025-11-11 00:44:22,887:test.notebook:INFO: [NOTEBOOK] Test message 3
2025-11-11 00:44:22,888:utils.portfolio.Portfolio:INFO: [MODULE] Test message from Portfolio module
2025-11-11 00:44:22,890:utils.portfolio.Portfolio:INFO: [MODULE] Second message from Portfolio
2025-11-11 00:44:22,891:utils.portfolio.Portfolio:INFO: [MODULE] Third message from Portfolio


ТЕСТ 1: Создание логгера НАПРЯМУЮ в ноутбуке (без модуля Portfolio)
Количество handlers у notebook logger: 2
  Handler 1: StreamHandler
  Handler 2: FileHandler

ТЕСТ 2: Вызов логгера из МОДУЛЯ Portfolio (через calc.logger)
Количество handlers у portfolio logger: 0

ТЕСТ 3: Проверка - это ОДИН И ТОТ ЖЕ объект логгера?
calc.logger is another_logger: False
ID calc.logger: 2003888488528
ID another_logger: 2003895677840
Handlers у another_logger: 2

⚠️ ПРОВЕРЬТЕ ВЫВОД ВЫШЕ:
  - Сколько раз появилось каждое сообщение?
  - Notebook messages должны появиться 1 раз
  - Module messages могут появиться несколько раз если есть дубликаты handlers


In [ ]:
# 🧪 ТЕСТ 4: Что происходит при ПОВТОРНОМ создании Portfolio без reload?
print("=" * 80)
print("ТЕСТ 4: Создаем ВТОРОЙ экземпляр Portfolio БЕЗ reload cell")
print("=" * 80)

# Создадим второй Portfolio без очистки handlers
calc2 = Portfolio(
    dt_calc='2025-05-31',
    dt_start='2019-11-03',
    stocks_step=10,
    tickers_list=tickers_list
)

# Проверяем handlers
portfolio_logger = logging.getLogger('utils.portfolio')
print(f"\nКоличество handlers после создания calc2: {len(portfolio_logger.handlers)}")
for i, h in enumerate(portfolio_logger.handlers):
    print(f"  Handler {i+1}: {type(h).__name__} (ID: {id(h)})")

# Тестируем логирование
print("\nЛогируем 2 сообщения через calc2.logger:")
calc2.logger.info("[CALC2] First message")
calc2.logger.info("[CALC2] Second message")

print("\n" + "=" * 80)
print("⚠️ ПРОВЕРЬТЕ:")
print("  - Если handlers добавились ДВА РАЗА - это проблема!")
print("  - Если handlers остались 2 штуки - всё работает правильно!")
print("=" * 80)


2025-11-11 00:44:27,787:utils.portfolio.Portfolio:INFO: Portfolio instance created with dt_calc=2025-05-31, dt_start=2019-11-03
2025-11-11 00:44:27,789:utils.portfolio.Portfolio:INFO: [CALC2] First message
2025-11-11 00:44:27,791:utils.portfolio.Portfolio:INFO: [CALC2] Second message


ТЕСТ 4: Создаем ВТОРОЙ экземпляр Portfolio БЕЗ reload cell

Количество handlers после создания calc2: 2
  Handler 1: StreamHandler (ID: 2003899582992)
  Handler 2: FileHandler (ID: 2003899593488)

Логируем 2 сообщения через calc2.logger:

⚠️ ПРОВЕРЬТЕ:
  - Если handlers добавились ДВА РАЗА - это проблема!
  - Если handlers остались 2 штуки - всё работает правильно!


In [ ]:
calc.portfolio.head(5)

,ticker,date,time,open,high,low,close,quarter,year,EV/EBITDA,...,P/FCF,P/S,debt,Долг/EBITDA,capitalization,interest_rate,inflation,Year,unemployment_rate,rubusd_exchange_rate
796,FESH,2019-12-31,000000,7.70,9.70,7.50,8.81,4,2019,4.83,...,NaN,0.46,3.290000e+10,2.65,2.600000e+10,0.0625,0.030,2019,0.046,61.9057
797,FESH,2020-01-31,000000,8.89,9.07,8.11,8.60,1,2020,NaN,...,NaN,NaN,3.290000e+10,NaN,2.600000e+10,0.0625,0.024,2020,0.058,63.0359
798,FESH,2020-02-29,000000,8.62,9.39,6.90,7.07,1,2020,NaN,...,NaN,NaN,3.290000e+10,NaN,2.600000e+10,0.0600,0.023,2020,0.058,66.9909
799,FESH,2020-03-31,000000,7.29,7.77,4.61,5.74,1,2020,NaN,...,NaN,NaN,3.290000e+10,NaN,2.600000e+10,0.0600,0.025,2020,0.058,77.7325
800,FESH,2020-04-30,000000,5.70,7.18,5.61,6.44,2,2020,4.80,...,25.2,0.38,3.320000e+10,2.81,2.140000e+10,0.0550,0.031,2020,0.058,73.6894


In [ ]:
calc.macro_connection_summary

### Ключевые результаты:

**1. Безработица** - наиболее значимый предиктор долга (73% компаний)
**2. USD/RUB** - сильно влияет на капитализацию (60% компаний) 
**3. Инфляция** - умеренное влияние, особенно на долг (60% vs 27%)

**"Доля значимых связей"** = процент компаний, для которых макроэкономический фактор **статистически значимо** влияет на капитализацию/долг.

In [ ]:
# Используем встроенный метод класса Portfolio
calc.plot_macro_significance(
    save_path='logs/graphs/macro_significance_summary.png',
    verbose=True,
    figsize=(10, 6)
)

ValueError: Macro connection summary not calculated. Run calc_macro_connections() first.

In [ ]:
def get_rubusd_exchange_rate(
    update_backup: bool=False,
    use_backup: bool=False
) -> pd.DataFrame:

    rubusd_df_path = f'data/macro/rubusd.csv'

    if use_backup:
        rates = pd.read_csv(rubusd_df_path)
        print(f'Exchange rates for usd/rub will be use from backup. Last actual date: {rates.date.max()}')
        return rates

    if os.path.exists(rubusd_df_path):

        rates = pd.read_csv(rubusd_df_path)
        start_date = rates.date.max()
    else:
        start_date = calc.dt_start

        date_range = pd.date_range(
            start=pd.to_datetime(start_date, format='%Y-%m-%d'),  # Исправлен формат на ISO8601
            end=pd.to_datetime(calc.dt_calc, format='%Y-%m-%d'),   # Исправлен формат на ISO8601
            freq='D'
        )

        rates_additional = []
        for date in tqdm(date_range[-10:]):
            rates_additional.append(
                (date.strftime('%Y.%m.%d'), float(ExchangeRates(date)['USD'].value))
            )

        rates = (
            pd.DataFrame(rates_additional, columns=['date', 'rubusd_exchange_rate'])
            if not os.path.exists(rubusd_df_path)
            else pd.concat(rates, pd.DataFrame(rates_additional, columns=rates.columns))
        )

    if update_backup:
        rates.to_csv(rubusd_df_path, index=False)
        print(f'Backup file for usd/rub exchange rates was updated. New actual date: {rates.date.max()}')

    return rates

In [ ]:
get_rubusd_exchange_rate(use_backup=True,update_backup=False)

Exchange rates for usd/rub will be use from backup. Last actual date: 2025-05-31


,date,rubusd_exchange_rate
0,2019-11-03,64.0316
1,2019-11-04,64.0316
2,2019-11-05,64.0316
3,2019-11-06,63.2480
4,2019-11-07,63.5880
...,...,...
2032,2025-05-27,79.6588
2033,2025-05-28,79.6176
2034,2025-05-29,79.6037
2035,2025-05-30,78.4970


In [ ]:
!cp logs/graphs/GAZP_pd.png text/img/GAZP_pd.png
!cp logs/graphs/irf_inflation_PD.png text/img/irf_inflation_PD.png
!cp logs/graphs/irf_interest_rate_PD.png text/img/irf_interest_rate_PD.png
!cp logs/graphs/corr_matrix.png text/img/corr_matrix.png

'cp' is not recognized as an internal or external command,
operable program or batch file.
'cp' is not recognized as an internal or external command,
operable program or batch file.
'cp' is not recognized as an internal or external command,
operable program or batch file.
'cp' is not recognized as an internal or external command,
operable program or batch file.
'cp' is not recognized as an internal or external command,
operable program or batch file.
'cp' is not recognized as an internal or external command,
operable program or batch file.
'cp' is not recognized as an internal or external command,
operable program or batch file.


## Анализ макроэкономических связей с помощью встроенного метода

Используем метод `calc_macro_connections` для статистического анализа влияния макроэкономических факторов на параметры модели Мертона (капитализация и долг компаний).

In [ ]:
# Создаем визуализацию результатов макроэкономического анализа
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))

# Подготавливаем данные для визуализации
factors = ['inflation', 'unemployment', 'usd_rub']
factor_names = {'inflation': 'Инфляция', 'unemployment': 'Безработица', 'usd_rub': 'USD/RUB'}

# 1. Средние коэффициенты для капитализации
cap_data = macro_summary[macro_summary['target'] == 'capitalization']
cap_coefs = [cap_data[f'coef_{factor}'].mean() for factor in factors]
colors_cap = ['red' if coef < 0 else 'green' for coef in cap_coefs]

bars1 = ax1.bar(range(len(factors)), cap_coefs, color=colors_cap, alpha=0.7)
ax1.set_title('Влияние макрофакторов на КАПИТАЛИЗАЦИЮ', fontsize=14, fontweight='bold')
ax1.set_ylabel('Средний коэффициент')
ax1.set_xticks(range(len(factors)))
ax1.set_xticklabels([factor_names[f] for f in factors])
ax1.axhline(y=0, color='black', linestyle='-', linewidth=0.8)
ax1.grid(True, alpha=0.3)

# Добавляем значения на столбцы
for i, (bar, coef) in enumerate(zip(bars1, cap_coefs)):
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height + (0.001 if height >= 0 else -0.003),
             f'{coef:.3f}', ha='center', va='bottom' if height >= 0 else 'top', fontweight='bold')

# 2. Средние коэффициенты для долга
debt_data = macro_summary[macro_summary['target'] == 'debt']
debt_coefs = [debt_data[f'coef_{factor}'].mean() for factor in factors]
colors_debt = ['red' if coef < 0 else 'green' for coef in debt_coefs]

bars2 = ax2.bar(range(len(factors)), debt_coefs, color=colors_debt, alpha=0.7)
ax2.set_title('Влияние макрофакторов на ДОЛГ', fontsize=14, fontweight='bold')
ax2.set_ylabel('Средний коэффициент')
ax2.set_xticks(range(len(factors)))
ax2.set_xticklabels([factor_names[f] for f in factors])
ax2.axhline(y=0, color='black', linestyle='-', linewidth=0.8)
ax2.grid(True, alpha=0.3)

# Добавляем значения на столбцы
for i, (bar, coef) in enumerate(zip(bars2, debt_coefs)):
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height + (0.05 if height >= 0 else -0.1),
             f'{coef:.2f}', ha='center', va='bottom' if height >= 0 else 'top', fontweight='bold')

# 3. Процент значимых связей для капитализации
cap_significance = []
for factor in factors:
    coef_col = f'coef_{factor}'
    ci_col = f'coef_{factor}_ci'
    significant_count = 0
    total_count = 0

    for _, row in cap_data.iterrows():
        if pd.notna(row[coef_col]):
            total_count += 1
            ci_str = row[ci_col]
            if pd.notna(ci_str) and isinstance(ci_str, str):
                import re
                ci_match = re.findall(r'-?\d+\.\d+', ci_str)
                if len(ci_match) == 2:
                    ci_low, ci_high = float(ci_match[0]), float(ci_match[1])
                    if (ci_low > 0 and ci_high > 0) or (ci_low < 0 and ci_high < 0):
                        significant_count += 1

    cap_significance.append(significant_count / total_count * 100 if total_count > 0 else 0)

bars3 = ax3.bar(range(len(factors)), cap_significance, color='skyblue', alpha=0.7)
ax3.set_title('Доля значимых связей (КАПИТАЛИЗАЦИЯ)', fontsize=14, fontweight='bold')
ax3.set_ylabel('Процент значимых связей, %')
ax3.set_xticks(range(len(factors)))
ax3.set_xticklabels([factor_names[f] for f in factors])
ax3.set_ylim(0, 100)
ax3.grid(True, alpha=0.3)

# Добавляем значения
for i, (bar, pct) in enumerate(zip(bars3, cap_significance)):
    height = bar.get_height()
    ax3.text(bar.get_x() + bar.get_width()/2., height + 2,
             f'{pct:.1f}%', ha='center', va='bottom', fontweight='bold')

# 4. Процент значимых связей для долга
debt_significance = []
for factor in factors:
    coef_col = f'coef_{factor}'
    ci_col = f'coef_{factor}_ci'
    significant_count = 0
    total_count = 0

    for _, row in debt_data.iterrows():
        if pd.notna(row[coef_col]):
            total_count += 1
            ci_str = row[ci_col]
            if pd.notna(ci_str) and isinstance(ci_str, str):
                import re
                ci_match = re.findall(r'-?\d+\.\d+', ci_str)
                if len(ci_match) == 2:
                    ci_low, ci_high = float(ci_match[0]), float(ci_match[1])
                    if (ci_low > 0 and ci_high > 0) or (ci_low < 0 and ci_high < 0):
                        significant_count += 1

    debt_significance.append(significant_count / total_count * 100 if total_count > 0 else 0)

bars4 = ax4.bar(range(len(factors)), debt_significance, color='orange', alpha=0.7)
ax4.set_title('Доля значимых связей (ДОЛГ)', fontsize=14, fontweight='bold')
ax4.set_ylabel('Процент значимых связей, %')
ax4.set_xticks(range(len(factors)))
ax4.set_xticklabels([factor_names[f] for f in factors])
ax4.set_ylim(0, 100)
ax4.grid(True, alpha=0.3)

# Добавляем значения
for i, (bar, pct) in enumerate(zip(bars4, debt_significance)):
    height = bar.get_height()
    ax4.text(bar.get_x() + bar.get_width()/2., height + 2,
             f'{pct:.1f}%', ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.savefig('logs/graphs/macro_connections_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

print("💾 График сохранен как: logs/graphs/macro_connections_analysis.png")

NameError: name 'plt' is not defined

In [ ]:
macro_summary.head(5)

,ticker,target,best_alpha,mse_model,mse_baseline,r2,coef_inflation,coef_inflation_ci,coef_unemployment,coef_unemployment_ci,coef_usd_rub,coef_usd_rub_ci
0,FESH,debt,100.000000,0.007767,0.008085,-0.008298,-0.001078,"[-0.004, 0.002]",0.004729,"[-0.002, 0.012]",-0.001864,"[-0.008, 0.004]"
1,FESH,capitalization,0.001000,0.097799,0.887980,0.889851,0.008658,"[-0.067, 0.133]",-0.716768,"[-0.823, -0.577]",0.038045,"[-0.056, 0.164]"
2,GAZP,debt,5.963623,0.003915,0.021115,0.814210,-0.009842,"[-0.025, 0.013]",-0.073216,"[-0.087, -0.055]",0.053612,"[0.037, 0.073]"
3,GAZP,capitalization,7.543120,0.045438,0.067220,0.323264,0.039113,"[-0.000, 0.075]",0.087186,"[0.047, 0.138]",-0.024845,"[-0.071, 0.020]"
4,GMKN,debt,1.456348,0.014184,0.023292,0.389610,-0.045151,"[-0.072, -0.019]",-0.060431,"[-0.100, -0.029]",0.074262,"[0.041, 0.106]"
